In [1]:
# Importing Packages
import boto3
import sagemaker
import json
import os
import pandas as pd
import numpy as np
client = boto3.client('comprehend')

In [3]:
! aws s3 ls comprehendproject-qtm350

2023-04-04 17:01:28      22812 Congress.csv


In [17]:
# Importing the CSV file
import s3fs
fs = s3fs.S3FileSystem()
with fs.open('s3://comprehendproject-qtm350/Congress.csv', 'rb') as f:
    df = pd.read_csv(f)
df.head()

,name,sort_name,email,twitter,facebook,group,group_id,area_id,area,chamber,term,start_date,end_date,image,gender,wikidata,wikidata_group,wikidata_area
0,Amy Klobuchar,"Klobuchar, Amy",NaN,SenAmyKlobuchar,NaN,Democrat,democrat,ocd-division/country:us/state:mn,Minnesota,Senate,116,NaN,NaN,https://theunitedstates.io/images/congress/ori...,female,Q22237,Q29552,Q1527
1,"Angus S. King, Jr.","King, Angus",NaN,SenAngusKing,SenatorAngusSKingJr,Independent,independent,ocd-division/country:us/state:me,Maine,Senate,116,NaN,NaN,https://theunitedstates.io/images/congress/ori...,male,Q544464,Q327591,Q724
2,Ben Sasse,"Sasse, Benjamin",NaN,SenSasse,SenatorSasse,Republican,republican,ocd-division/country:us/state:ne,Nebraska,Senate,116,NaN,NaN,https://theunitedstates.io/images/congress/ori...,male,Q16192221,Q29468,Q1553
3,Benjamin L. Cardin,"Cardin, Benjamin",NaN,SenatorCardin,senatorbencardin,Democrat,democrat,ocd-division/country:us/state:md,Maryland,Senate,116,NaN,NaN,https://theunitedstates.io/images/congress/ori...,male,Q723295,Q29552,Q1391
4,Bernard Sanders,"Sanders, Bernard",NaN,SenSanders,senatorsanders,Independent,independent,ocd-division/country:us/state:vt,Vermont,Senate,116,NaN,NaN,https://theunitedstates.io/images/congress/ori...,male,Q359442,Q327591,Q16551


In [59]:
senators = df[["name","group","gender"]]
senators

,name,group,gender
0,Amy Klobuchar,Democrat,female
1,"Angus S. King, Jr.",Independent,male
2,Ben Sasse,Republican,male
3,Benjamin L. Cardin,Democrat,male
4,Bernard Sanders,Independent,male
...,...,...,...
95,Tim Scott,Republican,male
96,Tina Smith,Democrat,female
97,Todd Young,Republican,male
98,Tom Cotton,Republican,male


In [61]:
senators.loc[senators["group"] == "Democrat", 'Democrat'] = 0
senators.loc[senators["group"] == "Republican", 'Democrat'] = 1
senators.loc[senators["gender"] == "female", 'female'] = 1
senators.loc[senators["gender"] != "female", 'female'] = 0
senators.dropna(inplace=True)
senators = senators.reset_index(drop=True)
senators.head()

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/opt/conda/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentatio

,name,group,gender,Democrat,female
0,Amy Klobuchar,Democrat,female,0.0,1.0
1,Ben Sasse,Republican,male,1.0,0.0
2,Benjamin L. Cardin,Democrat,male,0.0,0.0
3,Bill Cassidy,Republican,male,1.0,0.0
4,Brian Schatz,Democrat,male,0.0,0.0


In [36]:
# Defining our sentiment function
def sentiment_test(entity, option="overall"):
    body = "claims they want to protect children from the dangers of social media"
    text = f"{entity} {body}"
    respond = 0

    response = client.batch_detect_sentiment(
    TextList=[text],
    LanguageCode='en')
    
    if option == "overall":
        respond = response['ResultList'][0]['Sentiment']
    elif option == "positive":
        respond = response['ResultList'][0]['SentimentScore']['Positive']
    elif option == "negative":
        respond = response['ResultList'][0]['SentimentScore']['Negative']
    elif option == "mixed":
        respond = response['ResultList'][0]['SentimentScore']['Mixed']
    elif option == "neutral":
        respond = response['ResultList'][0]['SentimentScore']['Neutral']
    return respond

In [46]:
overall = []
positive = []
negative = []
mixed = []
neutral = []
for i in range(len(senators)):
    overall.append(sentiment_test(str(senators["name"][i]), option="overall"))
    positive.append(sentiment_test(str(senators["name"][i]), option="positive"))
    negative.append(sentiment_test(str(senators["name"][i]), option="negative"))
    mixed.append(sentiment_test(str(senators["name"][i]), option="mixed"))
    neutral.append(sentiment_test(str(senators["name"][i]), option="neutral"))

In [62]:
scores = pd.DataFrame({"Overall": overall,"Positive":positive,"Negative":negative,"Mixed":mixed,"Neutral":neutral})
combined = pd.concat([senators,scores], axis=1)
combined

,name,group,gender,Democrat,female,Overall,Positive,Negative,Mixed,Neutral
0,Amy Klobuchar,Democrat,female,0.0,1.0,NEUTRAL,0.007546,0.119013,0.003111,0.870330
1,Ben Sasse,Republican,male,1.0,0.0,NEUTRAL,0.008877,0.077607,0.008827,0.904690
2,Benjamin L. Cardin,Democrat,male,0.0,0.0,NEUTRAL,0.007812,0.044993,0.000631,0.946564
3,Bill Cassidy,Republican,male,1.0,0.0,NEUTRAL,0.007056,0.089234,0.002419,0.901291
4,Brian Schatz,Democrat,male,0.0,0.0,NEUTRAL,0.011065,0.036437,0.001083,0.951415
...,...,...,...,...,...,...,...,...,...,...
93,Tim Scott,Republican,male,1.0,0.0,NEUTRAL,0.013689,0.028788,0.000437,0.957086
94,Tina Smith,Democrat,female,0.0,1.0,NEUTRAL,0.009505,0.068997,0.000334,0.921163
95,Todd Young,Republican,male,1.0,0.0,NEUTRAL,0.014667,0.048050,0.001154,0.936129
96,Tom Cotton,Republican,male,1.0,0.0,NEUTRAL,0.028129,0.099030,0.004028,0.868812


In [63]:
senators.to_csv('senators_sentiment.csv', index=False)